In [2]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import sklearn
from scipy import ndimage, misc
import math
%matplotlib inline
import cv2
import time
import glob
from scipy.ndimage import gaussian_filter1d
from scipy import signal

img_list = []
filenamelist= []
for img in glob.glob("0525_images/CCD-3/*.jpg"):
    #img_list.append(cv2.imread(img,0))
    filenamelist.append(img)
np.array(filenamelist).shape

(152,)

In [3]:
threshold_1phase = 100
threshold_2phase_1 = 40#30
threshold_2phase_2 = 10#20
blur_size = 3
neighbor_degree = 5


for iter_file in range(len(filenamelist)):
    
    #==============read in file==============================
    start = time.time()
    filename = filenamelist[iter_file]
    print(filename)
    image = np.array(cv2.imread(filename,0))
    
    backtorgb = cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
    
    #==============use canny to find the contour
#     image_blur = cv2.blur(image,(9,9))
#     canny = cv2.Canny(image_blur,5,20)
    ret, thresh1 = cv2.threshold(image, 70, 255, cv2.THRESH_BINARY_INV)
    open_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(4,3))
    thresh1 = cv2.morphologyEx(thresh1, cv2.MORPH_OPEN, open_kernel)
    print(thresh1.shape)
    # ===========cut the ROI by the canny edge image================
    _ , contours, hierarchy = cv2.findContours(thresh1,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    approx_list = []
    index = []
    for i in range(np.array(contours).shape[0]):
        if (hierarchy[0][i][3] >= 0):
            if(cv2.contourArea(contours[i])>300000 and cv2.contourArea(contours[i])<500000):
                epsilon = 0.005# * cv2.arcLength(contours[i],True)
                index.append(i)
                approx = cv2.approxPolyDP(contours[i], epsilon, True)
                approx_list.append(approx)
    #==========================================find outer circle==============================================
    ret, thresh1 = cv2.threshold(image, 110, 255, cv2.THRESH_BINARY_INV)

    c,r = cv2.minEnclosingCircle(approx_list[0])
    r = int(r)
    cx = int(c[0])
    cy = int(c[1])
    # ===========mask the image to extract the inner circle================
    inner_contour_img = np.zeros_like(image)
    cv2.circle(inner_contour_img,(cx,cy),(r+7), (255,255,255),-1)
    #inner_contour_img = 1 - inner_contour_img
    oringmask = inner_contour_img
    image[oringmask == 0] = 255
    #=======optinal blur
    image = cv2.blur(image,(blur_size, blur_size))
    #============create 0.5 degree a line

    const = np.pi/180
    x = []
    y = []
    inner_x = []
    inner_y = []

    degree_delta = 0.5
    for degree in range(int(360//degree_delta)):

        degree_real = degree_delta*degree
        now_x = int((r+2)*np.sin(degree_real*const))+cx
        now_y = int((r+2)*np.cos(degree_real*const))+cy
        now_inner_x = int((r-30)*np.sin(degree_real*const))+cx
        now_inner_y = int((r-30)*np.cos(degree_real*const))+cy

        x.append(now_x)
        y.append(now_y)
        inner_x.append(now_inner_x)
        inner_y.append(now_inner_y)
        
        #============shot from center
        
    threshold_1phase = 100
    threshold_2phase = 10
    all_diff_list = []
    all_min_list = []
    degree_1phase_list = []
    for degree in range(int(360//degree_delta)):
        points_on_line = np.linspace((inner_x[degree], inner_y[degree]),(x[degree],y[degree]) ,200,dtype = int)
        pass_list = image[points_on_line[:,1],points_on_line[:,0]]
        
        #First use gaussian filter to make the curve smooth,and then find peaks and valley.
        pass_list = gaussian_filter1d(pass_list,4)
        peaks, _ = signal.find_peaks(pass_list, height=0, prominence = 10)
        valley, _ = signal.find_peaks(-pass_list, height=0, prominence = 10)
        peak_and_valley_list = np.concatenate((pass_list[peaks], pass_list[valley]), axis=0)
        
        #cacluate the difference of every peak-valley
        shift_peak_and_valley_list = np.concatenate((np.array([0]), peak_and_valley_list))
        peak_and_valley_list = np.concatenate((peak_and_valley_list,np.array([0])))
        diff_list = abs(peak_and_valley_list - shift_peak_and_valley_list)    
        diff_list = diff_list[1::2]
        
        if(len(peaks)==0 or len(valley)==0 ):#如果沒找到local min 和max
            all_diff_list.append(-1)
            all_min_list.append(-1)
            #cv2.line(backtorgb,(inner_x[degree],inner_y[degree]),(x[degree],y[degree]),255,2)
            #cv2.circle(backtorgb,(x[degree],y[degree]),30, (0,255 ,255), 5)
            continue
            
        now_diff = max(diff_list)
        all_diff_list.append(now_diff)
        all_min_list.append(min(pass_list[valley]))
        
        #如果valley pixel>120 或差值<某個值
        if((min(pass_list[valley])>120 or now_diff<threshold_1phase)): 
            degree_1phase_list.append(degree)
            
    #第一階段有問題的degree，第二階段看看是不是真的有問題        
    for i in range(len(degree_1phase_list)):
        degree = degree_1phase_list[i]
        now_valley_value = all_min_list[degree]
        prev_valley_value = all_min_list[(degree+neighbor_degree)%720]
        next_valley_value = all_min_list[(degree-neighbor_degree)%720]

        now_peak_valley_difference = all_diff_list[degree]
        prev_peak_valley_difference = all_diff_list[(degree+neighbor_degree)%int(360//degree_delta)]
        next_peak_valley_difference = all_diff_list[(degree-neighbor_degree)%int(360//degree_delta)]
        
        if(((float(now_valley_value)-float(prev_valley_value)) + (float(now_valley_value) - float(next_valley_value))>threshold_2phase_1)
           and ((float(prev_peak_valley_difference)-float(now_peak_valley_difference))+(float(next_peak_valley_difference)-float(now_peak_valley_difference)))>threshold_2phase_2):
            #cv2.line(backtorgb,(inner_x[degree],inner_y[degree]),(x[degree],y[degree]),255,2)
            cv2.circle(backtorgb,(x[degree],y[degree]),30, (0,255 ,255), 5)
    
    end = time.time()
    print(end - start)
    print(".\\Detect\\Stop3\\" +filenamelist[iter_file][18:-4]+'_Detect.jpg')
    cv2.imwrite(".\\Detect\\Stop3\\" +filenamelist[iter_file][18:-4]+'_Detect.jpg',backtorgb)
    

0525_images/CCD-3\1.jpg
(2048, 2448)
0.3420844078063965
.\Detect\Stop3\1_Detect.jpg
0525_images/CCD-3\10.jpg
(2048, 2448)
0.3151559829711914
.\Detect\Stop3\10_Detect.jpg
0525_images/CCD-3\100.jpg
(2048, 2448)
0.30817413330078125
.\Detect\Stop3\100_Detect.jpg
0525_images/CCD-3\101.jpg
(2048, 2448)
0.3221104145050049
.\Detect\Stop3\101_Detect.jpg
0525_images/CCD-3\102.jpg
(2048, 2448)
0.31516003608703613
.\Detect\Stop3\102_Detect.jpg
0525_images/CCD-3\103.jpg
(2048, 2448)
0.31415653228759766
.\Detect\Stop3\103_Detect.jpg
0525_images/CCD-3\104.jpg
(2048, 2448)
0.30817556381225586
.\Detect\Stop3\104_Detect.jpg
0525_images/CCD-3\105.jpg
(2048, 2448)
0.3101980686187744
.\Detect\Stop3\105_Detect.jpg
0525_images/CCD-3\106.jpg
(2048, 2448)
0.3201713562011719
.\Detect\Stop3\106_Detect.jpg
0525_images/CCD-3\107.jpg
(2048, 2448)
0.2932090759277344
.\Detect\Stop3\107_Detect.jpg
0525_images/CCD-3\108.jpg
(2048, 2448)
0.3121650218963623
.\Detect\Stop3\108_Detect.jpg
0525_images/CCD-3\109.jpg
(2048, 2

0.33806872367858887
.\Detect\Stop3\47_Detect.jpg
0525_images/CCD-3\48.jpg
(2048, 2448)
0.3181495666503906
.\Detect\Stop3\48_Detect.jpg
0525_images/CCD-3\49.jpg
(2048, 2448)
0.37100791931152344
.\Detect\Stop3\49_Detect.jpg
0525_images/CCD-3\5.jpg
(2048, 2448)
0.3341376781463623
.\Detect\Stop3\5_Detect.jpg
0525_images/CCD-3\50.jpg
(2048, 2448)
0.3201422691345215
.\Detect\Stop3\50_Detect.jpg
0525_images/CCD-3\51.jpg
(2048, 2448)
0.31516051292419434
.\Detect\Stop3\51_Detect.jpg
0525_images/CCD-3\52.jpg
(2048, 2448)
0.31113386154174805
.\Detect\Stop3\52_Detect.jpg
0525_images/CCD-3\53.jpg
(2048, 2448)
0.3261556625366211
.\Detect\Stop3\53_Detect.jpg
0525_images/CCD-3\54.jpg
(2048, 2448)
0.32815122604370117
.\Detect\Stop3\54_Detect.jpg
0525_images/CCD-3\55.jpg
(2048, 2448)
0.2992231845855713
.\Detect\Stop3\55_Detect.jpg
0525_images/CCD-3\56.jpg
(2048, 2448)
0.31911277770996094
.\Detect\Stop3\56_Detect.jpg
0525_images/CCD-3\57.jpg
(2048, 2448)
0.3141944408416748
.\Detect\Stop3\57_Detect.jpg
05